# VDEH Data Enrichment Pipeline

**Fokus:** Datenanreicherung über Deutsche Nationalbibliothek (DNB) API

## 🎯 Ziel
- Identifikation unvollständiger Datensätze
- Anreicherung fehlender Metadaten via DNB API (ISBN/ISSN)
- Konsistenzprüfung vorhandener Daten
- Validierung und Qualitätsverbesserung

## 📚 Input/Output
- **Input**: `data/vdeh/processed/03_language_detected_data.parquet`
- **Output**: `data/vdeh/processed/04_enriched_data.parquet`

## 🔗 API
- **DNB SRU API**: https://www.dnb.de/DE/Professionell/Metadatendienste/Datenbezug/SRU/sru_node.html
- **Abfrage**: ISBN/ISSN basierte Suche

In [1]:
# 🛠️ SETUP UND DATEN LADEN
import sys
from pathlib import Path
import time
import pandas as pd
import json

# Add project root to path
project_root = Path.cwd()
while not (project_root / 'config.yaml').exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root / 'src'))

from utils.notebook_utils import setup_notebook

project_root, config = setup_notebook()
print(f"✅ Project root: {project_root}")
print(f"✅ Project: {config.get('project.name')} v{config.get('project.version')}")

# DNB API laden
from dnb_api import DNB_SRU_BASE, query_dnb_by_isbn, query_dnb_by_issn, query_dnb_by_title_author
print(f"✅ DNB API Funktionen geladen")

2025-12-09 11:07:32 - utils.notebook_utils - INFO - Searching for project root...
2025-12-09 11:07:32 - utils.notebook_utils - INFO - Project root found: /media/sz/Data/Bibo/analysis
2025-12-09 11:07:32 - utils.notebook_utils - INFO - Loading configuration...
2025-12-09 11:07:32 - config_loader - INFO - Configuration loaded from /media/sz/Data/Bibo/analysis/config.yaml
2025-12-09 11:07:32 - utils.notebook_utils - INFO - Configuration loaded successfully: Dual-Source Bibliothek Bestandsvergleich


✅ Project root: /media/sz/Data/Bibo/analysis
✅ Project: Dual-Source Bibliothek Bestandsvergleich v2.0.0
✅ DNB API Funktionen geladen


In [2]:
# 📂 DATEN AUS VORHERIGER STUFE LADEN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
input_path = processed_dir / '03_language_detected_data.parquet'
metadata_path = processed_dir / '03_metadata.json'

if not input_path.exists():
    raise FileNotFoundError(f"Input-Datei nicht gefunden: {input_path}\n"
                          "Bitte führen Sie zuerst 03_vdeh_language_detection.ipynb aus.")

# Daten laden
df_vdeh = pd.read_parquet(input_path)

# Vorherige Metadaten laden
with open(metadata_path, 'r') as f:
    prev_metadata = json.load(f)

print(f"📂 Daten geladen aus: {input_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {list(df_vdeh.columns)}")
print(f"💾 Memory: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Quality Scores anzeigen
if 'completeness_score' in df_vdeh.columns:
    avg_completeness = df_vdeh['completeness_score'].mean()
    print(f"📊 Durchschnittliche Vollständigkeit: {avg_completeness:.1f}%")
    
    if 'quality_category' in df_vdeh.columns:
        quality_dist = df_vdeh['quality_category'].value_counts()
        print(f"📊 Qualitäts-Verteilung:")
        for cat, count in quality_dist.items():
            print(f"   {cat}: {count:,} ({count/len(df_vdeh)*100:.1f}%)")

📂 Daten geladen aus: /media/sz/Data/Bibo/analysis/data/vdeh/processed/03_language_detected_data.parquet
📊 Records: 58,305
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'pages', 'language', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation', 'isbn_valid', 'isbn_status', 'issn_valid', 'issn_status', 'detected_language', 'detected_language_confidence', 'detected_language_name']
💾 Memory: 54.5 MB


In [3]:
# 🔍 KANDIDATEN FÜR ANREICHERUNG IDENTIFIZIEREN
print("🔍 === KANDIDATEN-IDENTIFIKATION ===\n")

# Kriterien für Anreicherungskandidaten
enrichment_candidates = pd.DataFrame()

# 1. Alle Records mit ISBN (unabhängig von Vollständigkeit)
print("📋 Kriterium 1: ISBN vorhanden (Prüfung aller Datensätze mit ISBN)")

# ISBN-basierte Kandidaten - ALLE mit ISBN
if 'isbn' in df_vdeh.columns:
    has_isbn = df_vdeh['isbn'].notna()
    
    isbn_candidates = df_vdeh[has_isbn].copy()
    
    # Statistiken für Überblick
    missing_title = isbn_candidates['title'].isna()
    missing_authors = (isbn_candidates['authors_str'].isna()) | (isbn_candidates['authors_str'] == '')
    missing_year = isbn_candidates['year'].isna()
    
    print(f"   ISBN-Kandidaten (alle): {len(isbn_candidates):,}")
    print(f"     - Vollständige Datensätze: {(~missing_title & ~missing_authors & ~missing_year).sum():,}")
    print(f"     - Unvollständig (Titel): {missing_title.sum():,}")
    print(f"     - Unvollständig (Autoren): {missing_authors.sum():,}")
    print(f"     - Unvollständig (Jahr): {missing_year.sum():,}")

# ISSN-basierte Kandidaten - nur bei unvollständigen Metadaten
if 'issn' in df_vdeh.columns:
    has_issn = df_vdeh['issn'].notna()
    missing_title = df_vdeh['title'].isna()
    missing_authors = (df_vdeh['authors_str'].isna()) | (df_vdeh['authors_str'] == '')
    missing_year = df_vdeh['year'].isna()
    
    issn_candidates = df_vdeh[
        has_issn & (missing_title | missing_authors | missing_year)
    ].copy()
    
    print(f"\n   ISSN-Kandidaten (nur unvollständig): {len(issn_candidates):,}")
    print(f"     - Fehlender Titel: {issn_candidates['title'].isna().sum():,}")
    print(f"     - Fehlende Autoren: {(issn_candidates['authors_str'].isna() | (issn_candidates['authors_str'] == '')).sum():,}")
    print(f"     - Fehlendes Jahr: {issn_candidates['year'].isna().sum():,}")

# Kombiniere Kandidaten
all_candidates = pd.concat([isbn_candidates, issn_candidates]).drop_duplicates(subset=['id'])
print(f"\n✅ Gesamt Anreicherungs-Kandidaten (mit ISBN/ISSN): {len(all_candidates):,}")

# 2. Records OHNE ISBN aber MIT Titel + Autoren (für DNB-Suche via Titel/Autor)
no_isbn_but_searchable = df_vdeh[
    (df_vdeh['isbn'].isna()) &
    (df_vdeh['title'].notna()) &
    (df_vdeh['authors_str'].notna()) &
    (df_vdeh['authors_str'] != '')
].copy()

print(f"\n📋 Kriterium 2: Ohne ISBN aber mit Titel + Autoren (DNB Titel/Autor-Suche)")
print(f"   Kandidaten: {len(no_isbn_but_searchable):,}")
print(f"     - Mit Titel: {no_isbn_but_searchable['title'].notna().sum():,}")
print(f"     - Mit Autoren: {(no_isbn_but_searchable['authors_str'].notna() & (no_isbn_but_searchable['authors_str'] != '')).sum():,}")

# Finale Kandidatenliste (unique): ISBN-Kandidaten ODER Titel+Autor-Kandidaten
final_candidates = df_vdeh[
    # ENTWEDER: ISBN vorhanden (unabhängig von Vollständigkeit)
    (df_vdeh['isbn'].notna()) |
    # ODER: Kein ISBN aber Titel + Autoren vorhanden
    (
        (df_vdeh['isbn'].isna()) &
        (df_vdeh['title'].notna()) &
        (df_vdeh['authors_str'].notna()) &
        (df_vdeh['authors_str'] != '')
    )
].copy()

print(f"\n🎯 Finale Anreicherungs-Kandidaten: {len(final_candidates):,}")
print(f"   Mit ISBN: {final_candidates['isbn'].notna().sum():,}")
print(f"   Mit ISSN: {final_candidates['issn'].notna().sum():,}")

🔍 === KANDIDATEN-IDENTIFIKATION ===

📋 Kriterium 1: ISBN vorhanden (Prüfung aller Datensätze mit ISBN)
   ISBN-Kandidaten (alle): 10,507
     - Vollständige Datensätze: 7,810
     - Unvollständig (Titel): 0
     - Unvollständig (Autoren): 2,664
     - Unvollständig (Jahr): 87

   ISSN-Kandidaten (nur unvollständig): 671
     - Fehlender Titel: 0
     - Fehlende Autoren: 503
     - Fehlendes Jahr: 644

✅ Gesamt Anreicherungs-Kandidaten (mit ISBN/ISSN): 11,172

📋 Kriterium 2: Ohne ISBN aber mit Titel + Autoren (DNB Titel/Autor-Suche)
   Kandidaten: 9,645
     - Mit Titel: 9,645
     - Mit Autoren: 9,645

🎯 Finale Anreicherungs-Kandidaten: 20,152
   Mit ISBN: 10,507
   Mit ISSN: 213


In [4]:
# 🌐 DNB API STATUS
print("🌐 === DNB API STATUS ===\n")

print("✅ DNB API Funktionen aus src/dnb_api.py geladen")
print(f"   Base URL: {DNB_SRU_BASE}")
print(f"   Schema: MARC21-xml")
print(f"   Verfügbare Funktionen:")
print(f"     - query_dnb_by_isbn(isbn, max_records=1)")
print(f"     - query_dnb_by_issn(issn, max_records=1)")
print(f"     - query_dnb_by_title_author(title, author=None, max_records=1)")
print(f"     - query_dnb_by_title_year(title, year, max_records=1)")

🌐 === DNB API STATUS ===

✅ DNB API Funktionen aus src/dnb_api.py geladen
   Base URL: https://services.dnb.de/sru/dnb
   Schema: MARC21-xml
   Verfügbare Funktionen:
     - query_dnb_by_isbn(isbn, max_records=1)
     - query_dnb_by_issn(issn, max_records=1)
     - query_dnb_by_title_author(title, author=None, max_records=1)


In [5]:
# 🚀 DNB DATENABFRAGE
print("🚀 === DNB DATENABFRAGE ===\n")

# Konfiguration
RATE_LIMIT_DELAY = 1.0  # Sekunden zwischen Anfragen (erhöht von 0.5s wegen Timeouts)
SAVE_INTERVAL = 50  # Speichere alle N Abfragen
DNB_DATA_FILE = processed_dir / 'dnb_raw_data.parquet'

print(f"⚙️  Konfiguration:")
print(f"   Rate Limit: {RATE_LIMIT_DELAY}s pro Anfrage")
print(f"   Save Interval: Alle {SAVE_INTERVAL} Queries")
print(f"   Output: {DNB_DATA_FILE.name}")

# Lade vorhandene DNB-Daten (falls vorhanden)
if DNB_DATA_FILE.exists():
    print(f"\n📂 Lade vorhandene DNB-Daten...")
    dnb_data_df = pd.read_parquet(DNB_DATA_FILE)
    print(f"   Bereits abgefragt: {len(dnb_data_df):,}")
    print(f"   Davon erfolgreich: {(dnb_data_df['dnb_found'] == True).sum():,}")
else:
    print(f"\n📂 Keine vorhandenen DNB-Daten gefunden - starte neue Abfrage")
    dnb_data_df = pd.DataFrame(columns=[
        'vdeh_id', 'query_type', 'query_value',
        'dnb_found', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher',
        'dnb_isbn', 'dnb_issn'
    ])

# Sammle ISBN/ISSN aus Kandidaten
print(f"\n📋 Extrahiere ISBN/ISSN aus {len(final_candidates):,} Kandidaten...")

queries_isbn = final_candidates[final_candidates['isbn'].notna()][['id', 'isbn']].copy()
queries_isbn.columns = ['vdeh_id', 'query_value']
queries_isbn['query_type'] = 'ISBN'

queries_issn = final_candidates[
    final_candidates['isbn'].isna() & final_candidates['issn'].notna()
][['id', 'issn']].copy()
queries_issn.columns = ['vdeh_id', 'query_value']
queries_issn['query_type'] = 'ISSN'

all_queries = pd.concat([queries_isbn, queries_issn], ignore_index=True)

print(f"   ISBN-Queries: {len(queries_isbn):,}")
print(f"   ISSN-Queries: {len(queries_issn):,}")
print(f"   Gesamt: {len(all_queries):,}")

# Filtere bereits abgefragte ISBN/ISSN
if len(dnb_data_df) > 0:
    # Erstelle Set der bereits abgefragten query_values
    already_queried = set(dnb_data_df['query_value'])
    
    # Filtere nur neue Queries
    new_queries = all_queries[~all_queries['query_value'].isin(already_queried)].copy()
    
    print(f"\n🔍 Abgleich mit vorhandenen Daten:")
    print(f"   Bereits vorhanden: {len(all_queries) - len(new_queries):,}")
    print(f"   Neu abzufragen: {len(new_queries):,}")
else:
    new_queries = all_queries
    print(f"\n🔍 Alle {len(new_queries):,} Queries sind neu")

# Nur abfragen wenn neue Queries vorhanden
if len(new_queries) > 0:
    print(f"\n🔄 Starte DNB-Abfrage für {len(new_queries):,} neue Queries...\n")
    
    from tqdm.auto import tqdm
    
    results = []
    stats = {'found': 0, 'not_found': 0}
    query_count = 0
    
    for _, row in tqdm(new_queries.iterrows(), total=len(new_queries), desc="🔍 DNB API", unit="queries"):
        # API-Abfrage
        dnb_result = None
        if row['query_type'] == 'ISBN':
            dnb_result = query_dnb_by_isbn(row['query_value'])
        elif row['query_type'] == 'ISSN':
            dnb_result = query_dnb_by_issn(row['query_value'])
        
        # Ergebnis speichern (inkl. ISBN/ISSN aus DNB-Antwort!)
        result_row = {
            'vdeh_id': row['vdeh_id'],
            'query_type': row['query_type'],
            'query_value': row['query_value'],
            'dnb_found': dnb_result is not None,
            'dnb_title': dnb_result.get('title') if dnb_result else None,
            'dnb_authors': ', '.join(dnb_result.get('authors', [])) if dnb_result else None,
            'dnb_year': dnb_result.get('year') if dnb_result else None,
            'dnb_publisher': dnb_result.get('publisher') if dnb_result else None,
            'dnb_isbn': dnb_result.get('isbn') if dnb_result else None,
            'dnb_issn': dnb_result.get('issn') if dnb_result else None,
            'dnb_pages': dnb_result.get('pages') if dnb_result else None
        }
        
        results.append(result_row)
        
        if dnb_result:
            stats['found'] += 1
        else:
            stats['not_found'] += 1
        
        query_count += 1
        
        # Regelmäßiges Speichern (alle SAVE_INTERVAL Queries)
        if query_count % SAVE_INTERVAL == 0:
            # Merge mit vorhandenen Daten
            new_results_df = pd.DataFrame(results)
            dnb_data_df = pd.concat([dnb_data_df, new_results_df], ignore_index=True)
            
            # Speichern
            dnb_data_df.to_parquet(DNB_DATA_FILE, index=False)
            
            # Reset results für nächste Batch
            results = []
            
            print(f"💾 Zwischenspeicherung: {query_count}/{len(new_queries)} Queries abgefragt")
        
        # Rate Limiting
        time.sleep(RATE_LIMIT_DELAY)
    
    # Finale Speicherung (restliche Ergebnisse)
    if len(results) > 0:
        new_results_df = pd.DataFrame(results)
        dnb_data_df = pd.concat([dnb_data_df, new_results_df], ignore_index=True)
        dnb_data_df.to_parquet(DNB_DATA_FILE, index=False)
    
    print(f"\n💾 DNB-Daten gespeichert: {DNB_DATA_FILE.name}")
    
    # Zusammenfassung
    print(f"\n📊 === NEUE ABFRAGEN ===")
    print(f"   Neue Queries: {len(new_queries):,}")
    print(f"   ✅ Gefunden: {stats['found']:,} ({stats['found']/len(new_queries)*100:.1f}%)")
    print(f"   ❌ Nicht gefunden: {stats['not_found']:,} ({stats['not_found']/len(new_queries)*100:.1f}%)")
    print(f"   💾 Zwischenspeicherungen: {len(new_queries)//SAVE_INTERVAL}")

else:
    print(f"\n✅ Alle ISBN/ISSN bereits in DNB-Daten vorhanden - keine neuen Abfragen nötig")

# Gesamtstatistik
print(f"\n📊 === GESAMT DNB-DATEN ===")
print(f"   Total Records: {len(dnb_data_df):,}")
print(f"   Erfolgreich: {(dnb_data_df['dnb_found'] == True).sum():,}")
print(f"   Nicht gefunden: {(dnb_data_df['dnb_found'] == False).sum():,}")

# Neue Statistik: ISBN/ISSN-Gewinn
if 'dnb_isbn' in dnb_data_df.columns:
    isbn_from_dnb = (dnb_data_df['dnb_found'] == True) & dnb_data_df['dnb_isbn'].notna()
    issn_from_dnb = (dnb_data_df['dnb_found'] == True) & dnb_data_df['dnb_issn'].notna()
    print(f"   📚 Mit DNB-ISBN: {isbn_from_dnb.sum():,}")
    print(f"   📰 Mit DNB-ISSN: {issn_from_dnb.sum():,}")

print(f"\n✅ DNB-Daten verfügbar als: dnb_data_df")
print(f"   Shape: {dnb_data_df.shape}")

🚀 === DNB DATENABFRAGE ===

⚙️  Konfiguration:
   Rate Limit: 1.0s pro Anfrage
   Save Interval: Alle 50 Queries
   Output: dnb_raw_data.parquet

📂 Lade vorhandene DNB-Daten...
   Bereits abgefragt: 6,350
   Davon erfolgreich: 3,286

📋 Extrahiere ISBN/ISSN aus 20,152 Kandidaten...
   ISBN-Queries: 10,507
   ISSN-Queries: 194
   Gesamt: 10,701

🔍 Abgleich mit vorhandenen Daten:
   Bereits vorhanden: 6,386
   Neu abzufragen: 4,315

🔄 Starte DNB-Abfrage für 4,315 neue Queries...



🔍 DNB API:   0%|          | 0/4315 [00:00<?, ?queries/s]

💾 Zwischenspeicherung: 50/4315 Queries abgefragt
💾 Zwischenspeicherung: 100/4315 Queries abgefragt
💾 Zwischenspeicherung: 150/4315 Queries abgefragt
💾 Zwischenspeicherung: 200/4315 Queries abgefragt
💾 Zwischenspeicherung: 250/4315 Queries abgefragt
💾 Zwischenspeicherung: 300/4315 Queries abgefragt
💾 Zwischenspeicherung: 350/4315 Queries abgefragt
💾 Zwischenspeicherung: 400/4315 Queries abgefragt
💾 Zwischenspeicherung: 450/4315 Queries abgefragt
💾 Zwischenspeicherung: 500/4315 Queries abgefragt
💾 Zwischenspeicherung: 550/4315 Queries abgefragt
💾 Zwischenspeicherung: 600/4315 Queries abgefragt
💾 Zwischenspeicherung: 650/4315 Queries abgefragt
💾 Zwischenspeicherung: 700/4315 Queries abgefragt
💾 Zwischenspeicherung: 750/4315 Queries abgefragt
💾 Zwischenspeicherung: 800/4315 Queries abgefragt
💾 Zwischenspeicherung: 850/4315 Queries abgefragt
💾 Zwischenspeicherung: 900/4315 Queries abgefragt
💾 Zwischenspeicherung: 950/4315 Queries abgefragt
💾 Zwischenspeicherung: 1000/4315 Queries abgefragt


/tmp/ipykernel_19216/2756518088.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dnb_data_df = pd.concat([dnb_data_df, new_results_df], ignore_index=True)


💾 Zwischenspeicherung: 4200/4315 Queries abgefragt
💾 Zwischenspeicherung: 4250/4315 Queries abgefragt
💾 Zwischenspeicherung: 4300/4315 Queries abgefragt

💾 DNB-Daten gespeichert: dnb_raw_data.parquet

📊 === NEUE ABFRAGEN ===
   Neue Queries: 4,315
   ✅ Gefunden: 2,484 (57.6%)
   ❌ Nicht gefunden: 1,831 (42.4%)
   💾 Zwischenspeicherungen: 86

📊 === GESAMT DNB-DATEN ===
   Total Records: 10,665
   Erfolgreich: 5,770
   Nicht gefunden: 4,895
   📚 Mit DNB-ISBN: 5,770
   📰 Mit DNB-ISSN: 43

✅ DNB-Daten verfügbar als: dnb_data_df
   Shape: (10665, 10)


/tmp/ipykernel_19216/2756518088.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dnb_data_df = pd.concat([dnb_data_df, new_results_df], ignore_index=True)


In [6]:
dnb_data_df.head()

,vdeh_id,query_type,query_value,dnb_found,dnb_title,dnb_authors,dnb_year,dnb_publisher,dnb_isbn,dnb_issn
0,000000023,ISBN,3-428-05409-1,True,Die deutsche Roheisenindustrie 1871 - 1913,"Krengel, Jochen",1983.0,Duncker und Humblot,3428054091,None
1,000000038,ISBN,3-527-26070-6,True,Korrosionskunde im Experiment,"Heitz, Ewald, Henkhaus, Rolf, Rahmel, Alfred",1983.0,Verlag Chemie,3527260706,None
2,000000039,ISBN,3-802-74302-4,True,Brandschutz und Feuersicherheit in Arbeitsstä...,"Isterling, Fritz",1984.0,Vulkan-Verlag,3802743024,None
3,000000040,ISBN,3-802-70475-4,True,Lagerung von staubförmigen Schüttgütern in ...,"Koster, Karl H., Haus der Technik",1983.0,Vulkan-Verlag Classen,3802704754,None
4,000000042,ISBN,0-853-34164-8,False,None,None,NaN,None,None,None


In [7]:
print(f"{dnb_data_df.dnb_found.sum()} von {dnb_data_df.shape[0]} Datensätze bei DNB gefunden")

5770 von 10665 Datensätze bei DNB gefunden


In [8]:
# 🔍 DNB TITEL/AUTOR-SUCHE (mit verbesserter Suchstrategie!)
print("🔍 === DNB TITEL/AUTOR-SUCHE ===\n")

# Konfiguration
DNB_TITLE_DATA_FILE = processed_dir / 'dnb_title_author_data.parquet'
ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS = True  # Variante A: maximale Redundanz
RESET_TA_SEARCH = False  # Set to True to reset and re-run with improved strategy

print(f"⚙️  Konfiguration:")
print(f"   Rate Limit: {RATE_LIMIT_DELAY}s pro Anfrage")
print(f"   Save Interval: Alle {SAVE_INTERVAL} Queries")
print(f"   Output: {DNB_TITLE_DATA_FILE.name}")
print(f"   TA für alle Titel+Autoren: {ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS}")
print(f"   🔄 Verbesserter Suchalgorithmus aktiv (4-stufige Strategie)")

# Reset wenn gewünscht
if RESET_TA_SEARCH and DNB_TITLE_DATA_FILE.exists():
    backup_file = processed_dir / f'dnb_title_author_data_OLD_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.parquet'
    DNB_TITLE_DATA_FILE.rename(backup_file)
    print(f"\n🔄 TA-Suche wird zurückgesetzt - alte Datei gesichert: {backup_file.name}")

# Lade vorhandene Titel/Autor-Suchdaten (falls vorhanden)
if DNB_TITLE_DATA_FILE.exists():
    print(f"\n📂 Lade vorhandene Titel/Autor-Suchdaten...")
    dnb_title_df = pd.read_parquet(DNB_TITLE_DATA_FILE)
    print(f"   Bereits abgefragt: {len(dnb_title_df):,}")
    print(f"   Davon erfolgreich: {(dnb_title_df['dnb_found'] == True).sum():,}")
else:
    print(f"\n📂 Keine vorhandenen Titel/Autor-Suchdaten gefunden - starte neue Abfrage")
    dnb_title_df = pd.DataFrame(columns=[
        'vdeh_id', 'query_type', 'title', 'author',
        'dnb_found', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher',
        'dnb_isbn', 'dnb_issn'
    ])

# Identifiziere Kandidaten für Titel/Autor-Suche
if ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS:
    title_author_candidates = df_vdeh[
        (df_vdeh['title'].notna()) &
        (df_vdeh['authors_str'].notna()) &
        (df_vdeh['authors_str'] != '')
    ].copy()
else:
    title_author_candidates = df_vdeh[
        (df_vdeh['isbn'].isna()) &
        (df_vdeh['issn'].isna()) &
        (df_vdeh['title'].notna()) &
        (df_vdeh['authors_str'].notna()) &
        (df_vdeh['authors_str'] != '')
    ].copy()

print(f"\n📋 Titel/Autor-Kandidaten: {len(title_author_candidates):,}")
print(f"   Mit Titel: {title_author_candidates['title'].notna().sum():,}")
print(f"   Mit Autoren: {(title_author_candidates['authors_str'].notna() & (title_author_candidates['authors_str'] != '')).sum():,}")

# Erstelle Query-Liste
title_queries = title_author_candidates[['id', 'title', 'authors_str']].copy()
title_queries.columns = ['vdeh_id', 'title', 'author']
title_queries['query_type'] = 'TITLE_AUTHOR'

print(f"   Gesamt Titel/Autor-Queries (vor Deduplikation): {len(title_queries):,}")

# Filtere bereits abgefragte Titel/Autor-Kombinationen
if len(dnb_title_df) > 0:
    # Erstelle Set der bereits abgefragten vdeh_ids
    already_queried = set(dnb_title_df['vdeh_id'])
    
    # Filtere nur neue Queries
    new_title_queries = title_queries[~title_queries['vdeh_id'].isin(already_queried)].copy()
    
    print(f"\n🔍 Abgleich mit vorhandenen Daten:")
    print(f"   Bereits vorhanden: {len(title_queries) - len(new_title_queries):,}")
    print(f"   Neu abzufragen: {len(new_title_queries):,}")
else:
    new_title_queries = title_queries
    print(f"\n🔍 Alle {len(new_title_queries):,} Titel/Autor-Queries sind neu")

# Nur abfragen wenn neue Queries vorhanden
if len(new_title_queries) > 0:
    print(f"\n🔄 Starte DNB Titel/Autor-Abfrage für {len(new_title_queries):,} neue Queries...")
    print(f"   📚 Verwende verbesserte 4-stufige Suchstrategie:\n")
    print(f"      1. Titel (Phrase) + Autor")
    print(f"      2. Titel (Wörter) + Autor")
    print(f"      3. Nur Titel (Phrase)")
    print(f"      4. Nur Titel (Wörter)\n")
    
    from tqdm.auto import tqdm
    
    results = []
    stats = {'found': 0, 'not_found': 0}
    query_count = 0
    
    for _, row in tqdm(new_title_queries.iterrows(), total=len(new_title_queries), desc="🔍 DNB Titel/Autor", unit="queries"):
        # API-Abfrage (mit verbesserter Strategie + ISBN/ISSN-Extraktion!)
        dnb_result = query_dnb_by_title_author(row['title'], row['author'])
        
        # Ergebnis speichern (inkl. ISBN/ISSN!)
        result_row = {
            'vdeh_id': row['vdeh_id'],
            'query_type': row['query_type'],
            'title': row['title'],
            'author': row['author'],
            'dnb_found': dnb_result is not None,
            'dnb_title': dnb_result.get('title') if dnb_result else None,
            'dnb_authors': ', '.join(dnb_result.get('authors', [])) if dnb_result else None,
            'dnb_year': dnb_result.get('year') if dnb_result else None,
            'dnb_publisher': dnb_result.get('publisher') if dnb_result else None,
            'dnb_isbn': dnb_result.get('isbn') if dnb_result else None,
            'dnb_issn': dnb_result.get('issn') if dnb_result else None,
            'dnb_pages': dnb_result.get('pages') if dnb_result else None
        }
        
        results.append(result_row)
        
        if dnb_result:
            stats['found'] += 1
        else:
            stats['not_found'] += 1
        
        query_count += 1
        
        # Regelmäßiges Speichern (alle SAVE_INTERVAL Queries)
        if query_count % SAVE_INTERVAL == 0:
            # Merge mit vorhandenen Daten
            new_results_df = pd.DataFrame(results)
            dnb_title_df = pd.concat([dnb_title_df, new_results_df], ignore_index=True)
            
            # Speichern
            dnb_title_df.to_parquet(DNB_TITLE_DATA_FILE, index=False)
            
            # Reset results für nächste Batch
            results = []
            
            current_rate = stats['found'] / query_count * 100
            print(f"💾 Zwischenstand: {query_count}/{len(new_title_queries)} | Erfolgsrate: {current_rate:.1f}%")
        
        # Rate Limiting
        time.sleep(RATE_LIMIT_DELAY)
    
    # Finale Speicherung (restliche Ergebnisse)
    if len(results) > 0:
        new_results_df = pd.DataFrame(results)
        dnb_title_df = pd.concat([dnb_title_df, new_results_df], ignore_index=True)
        dnb_title_df.to_parquet(DNB_TITLE_DATA_FILE, index=False)
    
    print(f"\n💾 DNB Titel/Autor-Daten gespeichert: {DNB_TITLE_DATA_FILE.name}")
    
    # Zusammenfassung
    print(f"\n📊 === NEUE TITEL/AUTOR-ABFRAGEN ===")
    print(f"   Neue Queries: {len(new_title_queries):,}")
    print(f"   ✅ Gefunden: {stats['found']:,} ({stats['found']/len(new_title_queries)*100:.1f}%)")
    print(f"   ❌ Nicht gefunden: {stats['not_found']:,} ({stats['not_found']/len(new_title_queries)*100:.1f}%)")
    print(f"   💾 Zwischenspeicherungen: {len(new_title_queries)//SAVE_INTERVAL}")

else:
    print(f"\n✅ Alle Titel/Autor-Kombinationen bereits abgefragt - keine neuen Abfragen nötig")

# Gesamtstatistik
print(f"\n📊 === GESAMT TITEL/AUTOR-DATEN ===")
print(f"   Total Records: {len(dnb_title_df):,}")
print(f"   Erfolgreich: {(dnb_title_df['dnb_found'] == True).sum():,}")
print(f"   Nicht gefunden: {(dnb_title_df['dnb_found'] == False).sum():,}")
print(f"   📈 Erfolgsrate: {(dnb_title_df['dnb_found'] == True).sum()/len(dnb_title_df)*100:.1f}%")

# Neue Statistik: ISBN/ISSN-Gewinn via TA-Suche
if 'dnb_isbn' in dnb_title_df.columns:
    isbn_from_ta = (dnb_title_df['dnb_found'] == True) & dnb_title_df['dnb_isbn'].notna()
    issn_from_ta = (dnb_title_df['dnb_found'] == True) & dnb_title_df['dnb_issn'].notna()
    print(f"   📚 Mit DNB-ISBN (via TA): {isbn_from_ta.sum():,}")
    print(f"   📰 Mit DNB-ISSN (via TA): {issn_from_ta.sum():,}")

print(f"\n✅ Titel/Autor-Daten verfügbar als: dnb_title_df")
print(f"   Shape: {dnb_title_df.shape}")

🔍 === DNB TITEL/AUTOR-SUCHE ===

⚙️  Konfiguration:
   Rate Limit: 1.0s pro Anfrage
   Save Interval: Alle 50 Queries
   Output: dnb_title_author_data.parquet
   TA für alle Titel+Autoren: True
   🔄 Verbesserter Suchalgorithmus aktiv (4-stufige Strategie)

📂 Keine vorhandenen Titel/Autor-Suchdaten gefunden - starte neue Abfrage

📋 Titel/Autor-Kandidaten: 17,488
   Mit Titel: 17,488
   Mit Autoren: 17,488
   Gesamt Titel/Autor-Queries (vor Deduplikation): 17,488

🔍 Alle 17,488 Titel/Autor-Queries sind neu

🔄 Starte DNB Titel/Autor-Abfrage für 17,488 neue Queries...
   📚 Verwende verbesserte 4-stufige Suchstrategie:

      1. Titel (Phrase) + Autor
      2. Titel (Wörter) + Autor
      3. Nur Titel (Phrase)
      4. Nur Titel (Wörter)



🔍 DNB Titel/Autor:   0%|          | 0/17488 [00:00<?, ?queries/s]

/tmp/ipykernel_19216/1770775344.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dnb_title_df = pd.concat([dnb_title_df, new_results_df], ignore_index=True)


💾 Zwischenstand: 50/17488 | Erfolgsrate: 20.0%
💾 Zwischenstand: 100/17488 | Erfolgsrate: 28.0%
💾 Zwischenstand: 150/17488 | Erfolgsrate: 36.0%
💾 Zwischenstand: 200/17488 | Erfolgsrate: 45.0%
💾 Zwischenstand: 250/17488 | Erfolgsrate: 46.0%
💾 Zwischenstand: 300/17488 | Erfolgsrate: 44.7%
💾 Zwischenstand: 350/17488 | Erfolgsrate: 45.4%
💾 Zwischenstand: 400/17488 | Erfolgsrate: 47.5%
💾 Zwischenstand: 450/17488 | Erfolgsrate: 46.7%
💾 Zwischenstand: 500/17488 | Erfolgsrate: 46.6%
💾 Zwischenstand: 550/17488 | Erfolgsrate: 46.2%
💾 Zwischenstand: 600/17488 | Erfolgsrate: 45.2%
💾 Zwischenstand: 650/17488 | Erfolgsrate: 44.9%
💾 Zwischenstand: 700/17488 | Erfolgsrate: 47.1%
💾 Zwischenstand: 750/17488 | Erfolgsrate: 49.1%
💾 Zwischenstand: 800/17488 | Erfolgsrate: 48.6%
💾 Zwischenstand: 850/17488 | Erfolgsrate: 48.4%
💾 Zwischenstand: 900/17488 | Erfolgsrate: 49.1%
💾 Zwischenstand: 950/17488 | Erfolgsrate: 48.9%
💾 Zwischenstand: 1000/17488 | Erfolgsrate: 48.5%
💾 Zwischenstand: 1050/17488 | Erfolgsrat

2025-12-09 19:14:52 - dnb_api - WARNING - DNB query error for 'tit="Praktische Sozialpolitiker aus allen Ständen : : vom Throne bis zur Werkstätte, vom Palast bis zur Hütte" and per=Schütz': HTTPSConnectionPool(host='services.dnb.de', port=443): Read timed out. (read timeout=10)


💾 Zwischenstand: 15300/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15350/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15400/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15450/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15500/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15550/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15600/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15650/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15700/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15750/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15800/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15850/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15900/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 15950/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 16000/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 16050/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 16100/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 16150/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 16200/17488 | Erfolgsrate: 38.1%
💾 Zwischenstand: 16250/17488 | Erfolgsrate: 38.0%


In [ ]:
# 🔍 DNB TITEL/JAHR-SUCHE (Neue Methode für Records ohne ISBN/ISSN!)
print("🔍 === DNB TITEL/JAHR-SUCHE ===\n")

# Import new function
from dnb_api import query_dnb_by_title_year

# Konfiguration
DNB_TITLE_YEAR_DATA_FILE = processed_dir / 'dnb_title_year_data.parquet'
RESET_TY_SEARCH = False  # Set to True to reset and re-run

print(f"⚙️  Konfiguration:")
print(f"   Rate Limit: {RATE_LIMIT_DELAY}s pro Anfrage")
print(f"   Save Interval: Alle {SAVE_INTERVAL} Queries")
print(f"   Output: {DNB_TITLE_YEAR_DATA_FILE.name}")
print(f"   🆕 Neue Suchmethode: Titel + Jahr (für Records ohne ISBN/ISSN/Autoren)")

# Reset wenn gewünscht
if RESET_TY_SEARCH and DNB_TITLE_YEAR_DATA_FILE.exists():
    backup_file = processed_dir / f'dnb_title_year_data_OLD_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.parquet'
    DNB_TITLE_YEAR_DATA_FILE.rename(backup_file)
    print(f"\n🔄 TY-Suche wird zurückgesetzt - alte Datei gesichert: {backup_file.name}")

# Lade vorhandene Titel/Jahr-Suchdaten (falls vorhanden)
if DNB_TITLE_YEAR_DATA_FILE.exists():
    print(f"\n📂 Lade vorhandene Titel/Jahr-Suchdaten...")
    dnb_ty_df = pd.read_parquet(DNB_TITLE_YEAR_DATA_FILE)
    print(f"   Bereits abgefragt: {len(dnb_ty_df):,}")
    print(f"   Davon erfolgreich: {(dnb_ty_df['dnb_found'] == True).sum():,}")
else:
    print(f"\n📂 Keine vorhandenen Titel/Jahr-Suchdaten gefunden - starte neue Abfrage")
    dnb_ty_df = pd.DataFrame(columns=[
        'vdeh_id', 'query_type', 'title', 'year',
        'dnb_found', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher',
        'dnb_isbn', 'dnb_issn'
    ])

# Identifiziere Kandidaten für Titel/Jahr-Suche
# Nur Records OHNE ISBN/ISSN UND OHNE Autoren, aber MIT Titel und Jahr
title_year_candidates = df_vdeh[
    (df_vdeh['isbn'].isna()) &
    (df_vdeh['issn'].isna()) &
    ((df_vdeh['authors_str'].isna()) | (df_vdeh['authors_str'] == '')) &
    (df_vdeh['title'].notna()) &
    (df_vdeh['year'].notna())
].copy()

print(f"\n📋 Titel/Jahr-Kandidaten: {len(title_year_candidates):,}")
print(f"   Ohne ISBN/ISSN: {(title_year_candidates['isbn'].isna() & title_year_candidates['issn'].isna()).sum():,}")
print(f"   Ohne Autoren: {((title_year_candidates['authors_str'].isna()) | (title_year_candidates['authors_str'] == '')).sum():,}")
print(f"   Mit Titel: {title_year_candidates['title'].notna().sum():,}")
print(f"   Mit Jahr: {title_year_candidates['year'].notna().sum():,}")

# Erstelle Query-Liste
ty_queries = title_year_candidates[['id', 'title', 'year']].copy()
ty_queries.columns = ['vdeh_id', 'title', 'year']
ty_queries['query_type'] = 'TITLE_YEAR'

print(f"   Gesamt Titel/Jahr-Queries: {len(ty_queries):,}")

# Filtere bereits abgefragte Titel/Jahr-Kombinationen
if len(dnb_ty_df) > 0:
    already_queried = set(dnb_ty_df['vdeh_id'])
    new_ty_queries = ty_queries[~ty_queries['vdeh_id'].isin(already_queried)].copy()
    
    print(f"\n🔍 Abgleich mit vorhandenen Daten:")
    print(f"   Bereits vorhanden: {len(ty_queries) - len(new_ty_queries):,}")
    print(f"   Neu abzufragen: {len(new_ty_queries):,}")
else:
    new_ty_queries = ty_queries
    print(f"\n🔍 Alle {len(new_ty_queries):,} Titel/Jahr-Queries sind neu")

# Nur abfragen wenn neue Queries vorhanden
if len(new_ty_queries) > 0:
    print(f"\n🔄 Starte DNB Titel/Jahr-Abfrage für {len(new_ty_queries):,} neue Queries...")
    print(f"   📚 4-stufige Suchstrategie:\n")
    print(f"      1. Titel (Phrase) + exaktes Jahr")
    print(f"      2. Titel (Wörter) + exaktes Jahr")
    print(f"      3. Titel (Phrase) + Jahr ±1")
    print(f"      4. Titel (Wörter) + Jahr ±1\n")
    
    from tqdm.auto import tqdm
    
    results = []
    stats = {'found': 0, 'not_found': 0}
    query_count = 0
    
    for _, row in tqdm(new_ty_queries.iterrows(), total=len(new_ty_queries), desc="🔍 DNB Titel/Jahr", unit="queries"):
        # API-Abfrage
        dnb_result = query_dnb_by_title_year(row['title'], int(row['year']))
        
        # Ergebnis speichern
        result_row = {
            'vdeh_id': row['vdeh_id'],
            'query_type': row['query_type'],
            'title': row['title'],
            'year': row['year'],
            'dnb_found': dnb_result is not None,
            'dnb_title': dnb_result.get('title') if dnb_result else None,
            'dnb_authors': ', '.join(dnb_result.get('authors', [])) if dnb_result else None,
            'dnb_year': dnb_result.get('year') if dnb_result else None,
            'dnb_publisher': dnb_result.get('publisher') if dnb_result else None,
            'dnb_isbn': dnb_result.get('isbn') if dnb_result else None,
            'dnb_issn': dnb_result.get('issn') if dnb_result else None,
            'dnb_pages': dnb_result.get('pages') if dnb_result else None
        }
        
        results.append(result_row)
        
        if dnb_result:
            stats['found'] += 1
        else:
            stats['not_found'] += 1
        
        query_count += 1
        
        # Regelmäßiges Speichern
        if query_count % SAVE_INTERVAL == 0:
            new_results_df = pd.DataFrame(results)
            dnb_ty_df = pd.concat([dnb_ty_df, new_results_df], ignore_index=True)
            dnb_ty_df.to_parquet(DNB_TITLE_YEAR_DATA_FILE, index=False)
            results = []
            
            current_rate = stats['found'] / query_count * 100
            print(f"💾 Zwischenstand: {query_count}/{len(new_ty_queries)} | Erfolgsrate: {current_rate:.1f}%")
        
        # Rate Limiting
        time.sleep(RATE_LIMIT_DELAY)
    
    # Finale Speicherung
    if len(results) > 0:
        new_results_df = pd.DataFrame(results)
        dnb_ty_df = pd.concat([dnb_ty_df, new_results_df], ignore_index=True)
        dnb_ty_df.to_parquet(DNB_TITLE_YEAR_DATA_FILE, index=False)
    
    print(f"\n💾 DNB Titel/Jahr-Daten gespeichert: {DNB_TITLE_YEAR_DATA_FILE.name}")
    
    # Zusammenfassung
    print(f"\n📊 === NEUE TITEL/JAHR-ABFRAGEN ===" )
    print(f"   Neue Queries: {len(new_ty_queries):,}")
    print(f"   ✅ Gefunden: {stats['found']:,} ({stats['found']/len(new_ty_queries)*100:.1f}%)")
    print(f"   ❌ Nicht gefunden: {stats['not_found']:,} ({stats['not_found']/len(new_ty_queries)*100:.1f}%)")
    print(f"   💾 Zwischenspeicherungen: {len(new_ty_queries)//SAVE_INTERVAL}")

else:
    print(f"\n✅ Alle Titel/Jahr-Kombinationen bereits abgefragt - keine neuen Abfragen nötig")

# Gesamtstatistik
print(f"\n📊 === GESAMT TITEL/JAHR-DATEN ===" )
print(f"   Total Records: {len(dnb_ty_df):,}")
print(f"   Erfolgreich: {(dnb_ty_df['dnb_found'] == True).sum():,}")
print(f"   Nicht gefunden: {(dnb_ty_df['dnb_found'] == False).sum():,}")
if len(dnb_ty_df) > 0:
    print(f"   📈 Erfolgsrate: {(dnb_ty_df['dnb_found'] == True).sum()/len(dnb_ty_df)*100:.1f}%")

# ISBN/ISSN-Gewinn via TY-Suche
if 'dnb_isbn' in dnb_ty_df.columns and len(dnb_ty_df) > 0:
    isbn_from_ty = (dnb_ty_df['dnb_found'] == True) & dnb_ty_df['dnb_isbn'].notna()
    issn_from_ty = (dnb_ty_df['dnb_found'] == True) & dnb_ty_df['dnb_issn'].notna()
    authors_from_ty = (dnb_ty_df['dnb_found'] == True) & dnb_ty_df['dnb_authors'].notna()
    print(f"   📚 Mit DNB-ISBN (via TY): {isbn_from_ty.sum():,}")
    print(f"   📰 Mit DNB-ISSN (via TY): {issn_from_ty.sum():,}")
    print(f"   ✍️  Mit DNB-Autoren (via TY): {authors_from_ty.sum():,}")

print(f"\n✅ Titel/Jahr-Daten verfügbar als: dnb_ty_df")
print(f"   Shape: {dnb_ty_df.shape}")

In [9]:
dnb_title_df.head()

,vdeh_id,query_type,title,author,dnb_found,dnb_title,dnb_authors,dnb_year,dnb_publisher,dnb_isbn,dnb_issn
0,000000004,TITLE_AUTHOR,Untersuchung der Gleichgewichte zwischen flüss...,"Thielmann, R.",False,None,None,NaN,None,None,None
1,000000005,TITLE_AUTHOR,Electromagnetic stirring of steel during solid...,"Marr, H.S. | Ludlow, V. | Summers, C.",False,None,None,NaN,None,None,None
2,000000006,TITLE_AUTHOR,Optimierung der Schwingbeanspruchungen von Ant...,"Gudehus, H.",False,None,None,NaN,None,None,None
3,000000007,TITLE_AUTHOR,Optimierung der Schwingbeanspruchungen von Ant...,"Peuker, G. | Reimann, D.",False,None,None,NaN,None,None,None
4,000000008,TITLE_AUTHOR,Optimierung der Schwingbeanspruchungen von Ant...,"Wünsch, D. | Harmeyer, G. | John, F.",False,None,None,NaN,None,None,None


In [10]:
# 🔗 DNB-DATEN MIT VDEH-DATEN ZUSAMMENFÜHREN
print("🔗 === DNB-DATEN MERGE ===\n")

# Starte mit VDEH-Daten
df_enriched = df_vdeh.copy()

# 1. Merge ISBN/ISSN-basierte DNB-Daten (als ID-Variante)
if len(dnb_data_df) > 0:
    # Prüfe ob ISBN/ISSN-Spalten existieren (Rückwärtskompatibilität)
    cols_to_merge = ['vdeh_id', 'query_type', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher']
    if 'dnb_isbn' in dnb_data_df.columns:
        cols_to_merge.append('dnb_isbn')
    if 'dnb_issn' in dnb_data_df.columns:
        cols_to_merge.append('dnb_issn')
    
    dnb_isbn_issn = dnb_data_df[dnb_data_df['dnb_found'] == True][cols_to_merge].rename(
        columns={'query_type': 'dnb_query_method'}
    )
    
    df_enriched = df_enriched.merge(
        dnb_isbn_issn,
        left_on='id',
        right_on='vdeh_id',
        how='left',
        suffixes=('', '_dup')
    )
    if 'vdeh_id' in df_enriched.columns:
        df_enriched.drop(columns=['vdeh_id'], inplace=True)
    if 'dnb_title_dup' in df_enriched.columns:
        df_enriched.drop(columns=[c for c in df_enriched.columns if c.endswith('_dup')], inplace=True)
    
    print(f"✅ ISBN/ISSN-basierte DNB-Daten (ID) gemerged → Spalten: dnb_title, dnb_authors, dnb_year, dnb_publisher")
    if 'dnb_isbn' in df_enriched.columns:
        print(f"   + dnb_isbn, dnb_issn")
    print(f"   ID-Matches: {df_enriched['dnb_query_method'].notna().sum():,}")

# 2. Merge Titel/Autor-basierte DNB-Daten als separate Variante (_ta)
if len(dnb_title_df) > 0:
    # Prüfe ob ISBN/ISSN-Spalten existieren (Rückwärtskompatibilität)
    cols_to_merge_ta = ['vdeh_id', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher']
    if 'dnb_isbn' in dnb_title_df.columns:
        cols_to_merge_ta.append('dnb_isbn')
    if 'dnb_issn' in dnb_title_df.columns:
        cols_to_merge_ta.append('dnb_issn')
    
    dnb_title_matches = dnb_title_df[dnb_title_df['dnb_found'] == True][cols_to_merge_ta].copy()
    
    # Rename mit _ta Suffix
    rename_map = {
        'dnb_title': 'dnb_title_ta',
        'dnb_authors': 'dnb_authors_ta',
        'dnb_year': 'dnb_year_ta',
        'dnb_publisher': 'dnb_publisher_ta'
    }
    if 'dnb_isbn' in cols_to_merge_ta:
        rename_map['dnb_isbn'] = 'dnb_isbn_ta'
    if 'dnb_issn' in cols_to_merge_ta:
        rename_map['dnb_issn'] = 'dnb_issn_ta'
    if 'dnb_pages' in cols_to_merge_ta:
        rename_map['dnb_pages'] = 'dnb_pages_ta'
    
    dnb_title_matches = dnb_title_matches.rename(columns=rename_map)
    
    df_enriched = df_enriched.merge(
        dnb_title_matches,
        left_on='id',
        right_on='vdeh_id',
        how='left'
    )
    if 'vdeh_id' in df_enriched.columns:
        df_enriched.drop(columns=['vdeh_id'], inplace=True)
    
    print(f"✅ Titel/Autor-basierte DNB-Daten (TA) gemerged → Spalten: dnb_*_ta")
    if 'dnb_isbn_ta' in df_enriched.columns:
        print(f"   + dnb_isbn_ta, dnb_issn_ta")
    print(f"   TA-Matches: {df_enriched[['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']].notna().any(axis=1).sum():,}")

# 3. Rückwärtskompatibilität: dnb_query_method belassen (zeigt ID vs Titel/Autor als Primärquelle)
#    (wird künftig nur noch als Hinweis genutzt; Fusion vergleicht explizit beide Varianten)

# 3. Merge Titel/Jahr-basierte DNB-Daten als separate Variante (_ty)
if len(dnb_ty_df) > 0:
    # Prüfe ob ISBN/ISSN-Spalten existieren
    cols_to_merge_ty = ['vdeh_id', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher']
    if 'dnb_isbn' in dnb_ty_df.columns:
        cols_to_merge_ty.append('dnb_isbn')
    if 'dnb_issn' in dnb_ty_df.columns:
        cols_to_merge_ty.append('dnb_issn')

    dnb_ty_matches = dnb_ty_df[dnb_ty_df['dnb_found'] == True][cols_to_merge_ty].copy()

    # Rename mit _ty Suffix
    rename_map = {
        'dnb_title': 'dnb_title_ty',
        'dnb_authors': 'dnb_authors_ty',
        'dnb_year': 'dnb_year_ty',
        'dnb_publisher': 'dnb_publisher_ty'
    }
    if 'dnb_isbn' in cols_to_merge_ty:
        rename_map['dnb_isbn'] = 'dnb_isbn_ty'
    if 'dnb_issn' in cols_to_merge_ty:
        rename_map['dnb_issn'] = 'dnb_issn_ty'
    if 'dnb_pages' in cols_to_merge_ty:
        rename_map['dnb_pages'] = 'dnb_pages_ty'

    dnb_ty_matches = dnb_ty_matches.rename(columns=rename_map)

    df_enriched = df_enriched.merge(
        dnb_ty_matches,
        left_on='id',
        right_on='vdeh_id',
        how='left'
    )
    if 'vdeh_id' in df_enriched.columns:
        df_enriched.drop(columns=['vdeh_id'], inplace=True)

    print(f"✅ Titel/Jahr-basierte DNB-Daten (TY) gemerged → Spalten: dnb_*_ty")
    if 'dnb_isbn_ty' in df_enriched.columns:
        print(f"   + dnb_isbn_ty, dnb_issn_ty")
    print(f"   TY-Matches: {df_enriched[['dnb_title_ty','dnb_authors_ty','dnb_year_ty','dnb_publisher_ty']].notna().any(axis=1).sum():,}")

# 4. Rückwärtskompatibilität
#    dnb_query_method zeigt nur noch ID-Quelle; Fusion vergleicht alle drei Varianten (ID, TA, TY)

# # Zusammenfassung
print(f"\n📊 === MERGE ZUSAMMENFASSUNG ===")
print(f"   Total Records: {len(df_enriched):,}")
print(f"   Mit ID-DNB: {df_enriched['dnb_query_method'].notna().sum():,}")
print(f"   Mit TA-DNB: {df_enriched[['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']].notna().any(axis=1).sum():,}")
print(f"   Mit TY-DNB: {df_enriched[['dnb_title_ty','dnb_authors_ty','dnb_year_ty','dnb_publisher_ty']].notna().any(axis=1).sum() if 'dnb_title_ty' in df_enriched.columns else 0:,}")

# ISBN/ISSN-Statistik
if 'dnb_isbn' in df_enriched.columns:
    isbn_id_count = df_enriched['dnb_isbn'].notna().sum()
    isbn_ta_count = df_enriched.get('dnb_isbn_ta', pd.Series()).notna().sum() if 'dnb_isbn_ta' in df_enriched.columns else 0
    print(f"   Mit DNB-ISBN (ID): {isbn_id_count:,}")
    print(f"   Mit DNB-ISBN (TA): {isbn_ta_count:,}")
    
if 'dnb_issn' in df_enriched.columns:
    issn_id_count = df_enriched['dnb_issn'].notna().sum()
    issn_ta_count = df_enriched.get('dnb_issn_ta', pd.Series()).notna().sum() if 'dnb_issn_ta' in df_enriched.columns else 0
    print(f"   Mit DNB-ISSN (ID): {issn_id_count:,}")
    print(f"   Mit DNB-ISSN (TA): {issn_ta_count:,}")

if 'dnb_isbn_ty' in df_enriched.columns:
    isbn_ty_count = df_enriched['dnb_isbn_ty'].notna().sum()
    issn_ty_count = df_enriched.get('dnb_issn_ty', pd.Series()).notna().sum() if 'dnb_issn_ty' in df_enriched.columns else 0
    authors_ty_count = df_enriched['dnb_authors_ty'].notna().sum() if 'dnb_authors_ty' in df_enriched.columns else 0
    print(f"   Mit DNB-ISBN (TY): {isbn_ty_count:,}")
    print(f"   Mit DNB-ISSN (TY): {issn_ty_count:,}")
    print(f"   Mit DNB-Autoren (TY): {authors_ty_count:,}")

print(f"\n✅ df_enriched erstellt und bereit zum Speichern")

🔗 === DNB-DATEN MERGE ===

✅ ISBN/ISSN-basierte DNB-Daten (ID) gemerged → Spalten: dnb_title, dnb_authors, dnb_year, dnb_publisher
   + dnb_isbn, dnb_issn
   ID-Matches: 5,770
✅ Titel/Autor-basierte DNB-Daten (TA) gemerged → Spalten: dnb_*_ta
   + dnb_isbn_ta, dnb_issn_ta
   TA-Matches: 6,697

📊 === MERGE ZUSAMMENFASSUNG ===
   Total Records: 58,305
   Mit ID-DNB: 5,770
   Mit TA-DNB: 6,697
   Mit DNB-ISBN (ID): 5,770
   Mit DNB-ISBN (TA): 3,675
   Mit DNB-ISSN (ID): 43
   Mit DNB-ISSN (TA): 172

✅ df_enriched erstellt und bereit zum Speichern


In [11]:
# 🔧 DATENTYP-NORMALISIERUNG (VOR dem Speichern!)
print("🔧 === DATENTYP-NORMALISIERUNG ===\n")

# Konvertiere Jahr-Spalten zu Int64 (verhindert float/int Konflikte bei der Fusion)
year_columns = ['year', 'dnb_year', 'dnb_year_ta']

for col in year_columns:
    if col in df_enriched.columns:
        # Konvertiere zu Int64, NaN bleiben NaN
        original_count = df_enriched[col].notna().sum()
        df_enriched[col] = pd.to_numeric(df_enriched[col], errors='coerce').astype('Int64')
        new_count = df_enriched[col].notna().sum()
        
        print(f"   {col}: {original_count:,} → {new_count:,} (Int64)")
        
        if original_count != new_count:
            print(f"      ⚠️  {original_count - new_count:,} Werte konnten nicht konvertiert werden")

print(f"\n✅ Datentypen normalisiert - bereit zum Speichern")


🔧 === DATENTYP-NORMALISIERUNG ===

   year: 33,313 → 33,313 (Int64)
   dnb_year: 5,746 → 5,746 (Int64)
   dnb_year_ta: 5,195 → 5,195 (Int64)

✅ Datentypen normalisiert - bereit zum Speichern


In [12]:
# 💾 DATEN SPEICHERN (vor KI-Fusion)
print("💾 === DATEN SPEICHERN ===\n")

# Output-Pfade
output_path = processed_dir / '04_dnb_enriched_data.parquet'
metadata_output = processed_dir / '04_metadata.json'

# 1. Parquet speichern
df_enriched.to_parquet(output_path, index=False)
print(f"✅ DNB-angereicherte Daten gespeichert: {output_path.name}")
print(f"   Records: {len(df_enriched):,}")
print(f"   Spalten: {len(df_enriched.columns)}")
print(f"   Größe: {output_path.stat().st_size / 1024**2:.1f} MB")

# 2. Metadaten erstellen
metadata = {
    'step': '04_dnb_enrichment',
    'input_file': '03_language_detected_data.parquet',
    'output_file': '04_dnb_enriched_data.parquet',
    'timestamp': pd.Timestamp.now().isoformat(),
    'record_count': len(df_enriched),
    'columns': list(df_enriched.columns),
    
    # DNB Query Statistiken
    'dnb_queries': {
        'isbn_issn': {
            'total_queries': len(dnb_data_df) if len(dnb_data_df) > 0 else 0,
            'successful': int((dnb_data_df['dnb_found'] == True).sum()) if len(dnb_data_df) > 0 else 0,
            'failed': int((dnb_data_df['dnb_found'] == False).sum()) if len(dnb_data_df) > 0 else 0
        },
        'title_author': {
            'total_queries': len(dnb_title_df) if len(dnb_title_df) > 0 else 0,
            'successful': int((dnb_title_df['dnb_found'] == True).sum()) if len(dnb_title_df) > 0 else 0,
            'failed': int((dnb_title_df['dnb_found'] == False).sum()) if len(dnb_title_df) > 0 else 0
        },
        'title_year': {
            'total_queries': len(dnb_ty_df) if 'dnb_ty_df' in locals() and len(dnb_ty_df) > 0 else 0,
            'successful': int((dnb_ty_df['dnb_found'] == True).sum()) if 'dnb_ty_df' in locals() and len(dnb_ty_df) > 0 else 0,
            'failed': int((dnb_ty_df['dnb_found'] == False).sum()) if 'dnb_ty_df' in locals() and len(dnb_ty_df) > 0 else 0
        }
    },
    
    # DNB-Daten Verfügbarkeit (Varianten)
    'dnb_variants': {
        'id_available': int(df_enriched['dnb_query_method'].notna().sum()) if 'dnb_query_method' in df_enriched.columns else 0,
        'ta_available': int(df_enriched[['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']].notna().any(axis=1).sum()),
        'ty_available': int(df_enriched[['dnb_title_ty','dnb_authors_ty','dnb_year_ty','dnb_publisher_ty']].notna().any(axis=1).sum()) if 'dnb_title_ty' in df_enriched.columns else 0
    },
    
    # DNB-Feldverfügbarkeit
    'dnb_field_availability': {
        'id': {
            'title': int(df_enriched['dnb_title'].notna().sum()) if 'dnb_title' in df_enriched.columns else 0,
            'authors': int(df_enriched['dnb_authors'].notna().sum()) if 'dnb_authors' in df_enriched.columns else 0,
            'year': int(df_enriched['dnb_year'].notna().sum()) if 'dnb_year' in df_enriched.columns else 0,
            'publisher': int(df_enriched['dnb_publisher'].notna().sum()) if 'dnb_publisher' in df_enriched.columns else 0
        },
        'title_author': {
            'title': int(df_enriched['dnb_title_ta'].notna().sum()) if 'dnb_title_ta' in df_enriched.columns else 0,
            'authors': int(df_enriched['dnb_authors_ta'].notna().sum()) if 'dnb_authors_ta' in df_enriched.columns else 0,
            'year': int(df_enriched['dnb_year_ta'].notna().sum()) if 'dnb_year_ta' in df_enriched.columns else 0,
            'publisher': int(df_enriched['dnb_publisher_ta'].notna().sum()) if 'dnb_publisher_ta' in df_enriched.columns else 0
        },
        'title_year': {
            'title': int(df_enriched['dnb_title_ty'].notna().sum()) if 'dnb_title_ty' in df_enriched.columns else 0,
            'authors': int(df_enriched['dnb_authors_ty'].notna().sum()) if 'dnb_authors_ty' in df_enriched.columns else 0,
            'year': int(df_enriched['dnb_year_ty'].notna().sum()) if 'dnb_year_ty' in df_enriched.columns else 0,
            'publisher': int(df_enriched['dnb_publisher_ty'].notna().sum()) if 'dnb_publisher_ty' in df_enriched.columns else 0
        }
    },
    
    # Originaldaten-Vollständigkeit
    'vdeh_completeness': {
        'title': int(df_enriched['title'].notna().sum()),
        'authors': int((df_enriched['authors_str'].notna() & (df_enriched['authors_str'] != '')).sum()),
        'year': int(df_enriched['year'].notna().sum()),
        'publisher': int(df_enriched['publisher'].notna().sum())
    }
}

# Metadaten speichern
with open(metadata_output, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"\n✅ Metadaten gespeichert: {metadata_output.name}")

# 3. Zusammenfassung
print(f"\n📊 === DNB ENRICHMENT ABGESCHLOSSEN ===")
print(f"   Input: {len(df_vdeh):,} VDEH Records")
print(f"   Output: {len(df_enriched):,} Records mit DNB-Daten")
print(f"   DNB-Queries: {metadata['dnb_queries']['isbn_issn']['total_queries'] + metadata['dnb_queries']['title_author']['total_queries']:,}")
print(f"   ID-Variante verfügbar: {metadata['dnb_variants']['id_available']:,}")
print(f"   TA-Variante verfügbar: {metadata['dnb_variants']['ta_available']:,}")
print(f"   TY-Variante verfügbar: {metadata['dnb_variants']['ty_available']:,}")

print(f"\n➡️  Nächster Schritt: 05_vdeh_data_fusion.ipynb")
print(f"   KI-gestützte Fusion von VDEH und DNB Daten (alle drei Varianten: ID, TA, TY)")

print(f"\n🎉 DNB Enrichment erfolgreich abgeschlossen!")

💾 === DATEN SPEICHERN ===

✅ DNB-angereicherte Daten gespeichert: 04_dnb_enriched_data.parquet
   Records: 58,305
   Spalten: 34
   Größe: 5.0 MB

✅ Metadaten gespeichert: 04_metadata.json

📊 === DNB ENRICHMENT ABGESCHLOSSEN ===
   Input: 58,305 VDEH Records
   Output: 58,305 Records mit DNB-Daten
   DNB-Queries: 28,153
   ID-Variante verfügbar: 5,770
   TA-Variante verfügbar: 6,697

➡️  Nächster Schritt: 05_vdeh_data_fusion.ipynb
   KI-gestützte Fusion von VDEH und DNB Daten (beide Varianten)

🎉 DNB Enrichment erfolgreich abgeschlossen!
